<a href="https://colab.research.google.com/github/Kewton/kewton.blog.colab/blob/master/%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0_%3EDatabase%E3%82%B5%E3%83%B3%E3%83%97%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ① スクレイピングして table 要素を dataframe に格納する

## スクレイピング基底クラスの定義

In [1]:
from bs4 import BeautifulSoup
import requests
import itertools
import time
from tqdm import notebook as tqdm

class BaseScraping:
  '''
  スクレイピングを実行する基底クラスです。
  _url_list に格納された URL を実行して html ドキュメントを取得し
  htmldoc2data を使用して必要なデータを取得します。

  Note:
    抽象メソッド speedbuffer と htmldoc2data を継承して使用してください。
  '''

  HEADERS = {
    "User-Agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
  }

  def __init__(self, _url_list:list):
    self._url_list = _url_list

  def run(self):
    results = {}
    for _url in tqdm.tqdm(self._url_list):
      print(_url)
      
      try:
        _data = self.htmldoc2data(self.get_html_doc(_url))
        results[_url] = _data
      except Exception as e:
        print("例外args:", e.args)
      
      self.speedbuffer()
    return results

  def speedbuffer(self):
    _sleeptime = 3
    print("wait " + str(_sleeptime) + "[sec]")
    time.sleep(_sleeptime)
    return

  def get_html_doc(self, _url):
    html_doc = requests.get(_url, headers=self.HEADERS, stream=True).content
    return html_doc

  def htmldoc2data(self, _html_doc):
    '''
    継承してください
    '''
    return _html_doc

## table 要素を DtaFrame に格納するクラスの定義

In [2]:
import re
from bs4 import BeautifulSoup
import pandas as pd


class GetTableDataFromWeb(BaseScraping):
  '''
  スクレイピングのサンプルクラスです。
  テーブルに格納されている値を取得します
  '''

  def __init__(self, _url_list:list, _css_selector):
    '''
    _css_selector：ターゲットとなるテーブル要素のcssセレクタを指定します
    '''
    super().__init__(_url_list)
    self.__css_selector= _css_selector

  def remove_blank_line_and_blank(self, _text):
    # 空白行削除
    text = _text.strip()
    text = re.sub(r'\n\n','\n',text)

    # 間の空白を削除
    text = re.sub(r'[ ]*','',text)
    
    return text

  def htmldoc2data(self, _html_doc):
    soup = BeautifulSoup(_html_doc, "html.parser")

    # テーブルを取得
    _table = soup.select_one(self.__css_selector)

    # theadタグを探す
    thead = _table.find('thead') 

    # tbodyタグを探す
    tbody = _table.find('tbody') 

    if thead is not None and tbody is not None:
      # パターン①：thead と tbody タグが存在する場合

      # ヘッダーを取得
      ths = thead.tr.find_all('th')

      # DataFrame のヘッダーを作成
      columns = []
      for th in ths:
        columns.append(th.text)
      df = pd.DataFrame(columns=columns)

      # body を取得
      trs = tbody.find_all('tr')
      for tr in trs:
        # 各レコードからテキストを取得し、空白行や余計な空白を削除してDataFrame に格納して返却
        tdlist = []
        for td in tr.find_all('td'):
          tdlist.append(self.remove_blank_line_and_blank(td.text))
        sr = pd.Series(tdlist, index=df.columns)
        df = df.append(sr, ignore_index=True)
      return df
    else:
      # パターン②：thead と tbody タグのいずれかが存在しない場合

      df = pd.DataFrame()
      trs = _table.find_all('tr')

      for tr in trs:
        # 各レコードからテキストを取得し、空白行や余計な空白を削除してDataFrame に格納して返却
        tdlist = []
        for td in tr.find_all('th'):
          tdlist.append(self.remove_blank_line_and_blank(td.text))

        for td in tr.find_all('td'):
          tdlist.append(self.remove_blank_line_and_blank(td.text))

        if len(tdlist) > 0:
          sr = pd.Series(tdlist)
          df = df.append(sr, ignore_index=True)
      return df
  

## サンプルクラスの実行

In [3]:
# フェブラリーステークスの結果
_url_list =[
  "https://keiba.yahoo.co.jp/race/result/2205010811/",
  "https://keiba.yahoo.co.jp/race/result/2105010811/",
  "https://keiba.yahoo.co.jp/race/result/2005010811/"
]

getTableDataFromWeb = GetTableDataFromWeb(_url_list, "#raceScore")

results = getTableDataFromWeb.run()

print("============================")
print(_url_list[0])
results[_url_list[0]].head(5)

  0%|          | 0/3 [00:00<?, ?it/s]

https://keiba.yahoo.co.jp/race/result/2205010811/
wait 3[sec]
https://keiba.yahoo.co.jp/race/result/2105010811/
wait 3[sec]
https://keiba.yahoo.co.jp/race/result/2005010811/
wait 3[sec]
https://keiba.yahoo.co.jp/race/result/2205010811/


,着順,枠番,馬番,馬名性齢⁄馬体重⁄B,タイム(着差),通過順位上3Fタイム,騎手,人気(オッズ),調教師
0,1,3,6,カフェファラオ\n牡5/526(+6)/,1.33.8,04-0334.3,福永祐一\n57.0,2(5.1),堀宣行
1,2,8,15,テイエムサウスダン\n牡5/546(-8)/,1.34.221/2馬身,01-0135.0,岩田康誠\n57.0,5(8.9),飯田雄三
2,3,6,11,ソダシ\n牝4/476(+6)/,1.34.31/2馬身,02-0234.9,吉田隼人\n55.0,4(8.2),須貝尚介
3,4,7,13,ソリストサンダー\n牡7/486(0)/,1.34.3クビ,05-0634.6,戸崎圭太\n57.0,6(9),高柳大輔
4,5,4,7,タイムフライヤー\n牡7/482(+2)/,1.34.511/4馬身,10-1034.6,横山武史\n57.0,15(61.6),橋口慎介


# ② ①の結果をDataBaseに登録する

In [4]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## DBセッションの作成

In [5]:
_colab_dir = "/content/drive/MyDrive/Colab Notebooks/"
db_path = 'sqlite:///' + _colab_dir + 'scraping.sqlite3.db'

In [6]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
engine = create_engine(db_path)
SessionClass = sessionmaker(engine)  # セッションを作るクラスを作成
session = SessionClass()

## データの蓄積

In [7]:
for _url in _url_list:
  df = results[_url]
  df["URL"] = _url
  df.to_sql('load_data', engine, index=False, if_exists='append')

# ③ 蓄積したデータの取り出し

In [8]:
import pandas as pd
# 1〜3着を取得
_query = """\
select * from load_data where 着順 in ('1','2','3')
"""
df2 = pd.read_sql(_query, session.bind)
df2.head(60)

,着順,枠番,馬番,馬名性齢⁄馬体重⁄B,タイム(着差),通過順位上3Fタイム,騎手,人気(オッズ),調教師,URL
0,1,3,6,カフェファラオ\n牡5/526(+6)/,1.33.8,04-0334.3,福永祐一\n57.0,2(5.1),堀宣行,https://keiba.yahoo.co.jp/race/result/2205010811/
1,2,8,15,テイエムサウスダン\n牡5/546(-8)/,1.34.221/2馬身,01-0135.0,岩田康誠\n57.0,5(8.9),飯田雄三,https://keiba.yahoo.co.jp/race/result/2205010811/
2,3,6,11,ソダシ\n牝4/476(+6)/,1.34.31/2馬身,02-0234.9,吉田隼人\n55.0,4(8.2),須貝尚介,https://keiba.yahoo.co.jp/race/result/2205010811/
3,1,2,3,カフェファラオ\n牡4/514(+10)/,1.34.4,03-0335.6,C.ルメール\n57.0,1(3.3),堀宣行,https://keiba.yahoo.co.jp/race/result/2105010811/
4,2,5,10,エアスピネル\n牡8/488(-4)/,1.34.53/4馬身,10-0935.2,鮫島克駿\n57.0,9(28),笹田和秀,https://keiba.yahoo.co.jp/race/result/2105010811/
5,3,4,7,ワンダーリーデル\n牡8/534(+2)/B,1.34.813/4馬身,07-0435.7,横山典弘\n57.0,8(19.3),安田翔伍,https://keiba.yahoo.co.jp/race/result/2105010811/
6,1,6,12,モズアスコット\n牡6/494(0)/,1.35.2,08-0835.4,C.ルメール\n57.0,1(2.8),矢作芳人,https://keiba.yahoo.co.jp/race/result/2005010811/
7,2,8,15,ケイティブレイブ\n牡7/524(+4)/,1.35.621/2馬身,09-0935.6,長岡禎仁\n57.0,16(142.6),杉山晴紀,https://keiba.yahoo.co.jp/race/result/2005010811/
8,3,5,9,サンライズノヴァ\n牡6/534(+10)/,1.35.811/4馬身,12-1335.3,松山弘平\n57.0,3(6.8),音無秀孝,https://keiba.yahoo.co.jp/race/result/2005010811/


# 【ご参考】データ登録後 index(SQLite の場合) を追加する

In [9]:
import time
import sqlite3

class StopWatch:
    '''
    処理時間計測
    '''
    def sw_start(self):
      self.__starttime = time.time()
      return
    
    def sw_stop(self):
      print("■処理時間：")
      print(f"{time.time() - self.__starttime}[sec]")
      return 

class Sqlite3Access:
  # 必要に応じて変更して下さい
  __colab_dir = "/content/drive/MyDrive/Colab Notebooks/"
  
  def __init__(self, _db_name):
    self.db_name = _db_name
    self.sw = StopWatch()
    return
  
  def _connect(self):
    '''
    すでに存在すればコネクションをはる
    存在しなければ新たに作成する
    '''
    self.conn = sqlite3.connect(self.__colab_dir + self.db_name + ".db")
    return

  def _getCursor(self):
    '''
    sqliteを操作するためのカーソルを取得する
    '''
    self._connect()
    return self.conn.cursor()

  def select(self, _sql):
    '''
    select
    '''
    # SQLiteを操作するためのカーソルを作成
    cur = self._getCursor()
    try:
      # データ検索
      cur.execute(_sql)
    except Exception as e:
      print("例外args:", e.args)
      print("コネクションをクローズしました")
      self.conn.close()
      return
    
    # 大量データの取り出しを考慮してカーソルのままretrunする
    return cur

  def execute(self, _sql) -> bool:
    '''
    create
    insert
    updaet
    delete
    '''
    # SQLiteを操作するためのカーソルを作成
    cur = self._getCursor()

    try:
      self.sw.sw_start()
      
      # データ登録
      cur.execute(_sql)

      # コミットしないと登録が反映されない
      self.conn.commit()
    except Exception as e:
      self.sw.sw_stop()
      print("例外args:", e.args)
      print("コネクションをクローズしました")
      self.conn.close()
      return False
    
    self.sw.sw_stop()
    self.conn.close()
    return True 

  def executemany(self, _list, _sql):
    '''
    create
    insert
    updaet
    delete
    '''
    # SQLiteを操作するためのカーソルを作成
    cur = self._getCursor()

    try:
      self.sw.sw_start()
      
      # 複数データ登録
      cur.executemany(_sql, _list)

      # コミットしないと登録が反映されない
      self.conn.commit()
    except Exception as e:
      self.sw.sw_stop()
      print("例外args:", e.args)
      print("コネクションをクローズしました")
      self.conn.close()
      return False
    self.sw.sw_stop()
    self.conn.close()
    return True
  
  def printSelect(self, _sql):
    '''
    select結果をコンソールに出力する
    '''
    print("■実行SQL：")
    print(_sql)
    print("■実行結果：")
    self.sw.sw_start()
    for row in self.select(_sql):
        print(row)
    self.sw.sw_stop()

In [10]:
scrapingdb = Sqlite3Access("scraping.sqlite3")
scrapingdb.db_name

'scraping.sqlite3'

In [11]:
# select文を作成
_sql =  """\
select * from sqlite_master;
"""

# select分を実行
scrapingdb.printSelect(_sql)

■実行SQL：
select * from sqlite_master;

■実行結果：
('table', 'load_data', 'load_data', 2, 'CREATE TABLE load_data (\n\t"着順" TEXT, \n\t"枠番" TEXT, \n\t"馬番" TEXT, \n\t"馬名性齢⁄馬体重⁄B" TEXT, \n\t"タイム(着差)" TEXT, \n\t"通過順位上3Fタイム" TEXT, \n\t"騎手" TEXT, \n\t"人気(オッズ)" TEXT, \n\t"調教師" TEXT, \n\t"URL" TEXT\n)')
■処理時間：
0.005756378173828125[sec]


## index 登録

In [12]:
# create文を作成
_sql =  """\
CREATE INDEX ix_001 ON load_data (URL, 着順);
"""

# create文を実行
scrapingdb.execute(_sql)

■処理時間：
0.016704082489013672[sec]


True

## index 登録結果の確認

In [13]:
# select文を作成
_sql =  """\
select * from sqlite_master;
"""

# select分を実行
scrapingdb.printSelect(_sql)

■実行SQL：
select * from sqlite_master;

■実行結果：
('table', 'load_data', 'load_data', 2, 'CREATE TABLE load_data (\n\t"着順" TEXT, \n\t"枠番" TEXT, \n\t"馬番" TEXT, \n\t"馬名性齢⁄馬体重⁄B" TEXT, \n\t"タイム(着差)" TEXT, \n\t"通過順位上3Fタイム" TEXT, \n\t"騎手" TEXT, \n\t"人気(オッズ)" TEXT, \n\t"調教師" TEXT, \n\t"URL" TEXT\n)')
('index', 'ix_001', 'load_data', 5, 'CREATE INDEX ix_001 ON load_data (URL, 着順)')
■処理時間：
0.004522800445556641[sec]
